#### Draft Use Cases

##### Imports and Global Variables

Run this cell before any of the others. It imports packages and sets variables that will be used throughout the notebook.

In [ ]:
import requests
import json
sasserver = "https://your_server"
print (sasserver)
prefix = "Example"

##### Get Access Token <a id='authentication'></a>

- This use case sets the `access_token` that will be used in all of the calls in this use case. Edit the payload object with a username and password. Run this as needed because the token will expire after a certain amount of time.

In [ ]:
url = sasserver + "/SASLogon/oauth/token#password"

payload={
    'grant_type': 'password',
    'username': 'your_username',
    'password': 'your_password'
}
files=[

]
headers = {
  'Authorization': 'Basic c2FzLmVjOg==',
}

response = requests.request("POST", url, headers=headers, data=payload, files=files, verify=False)


if response.status_code == 200:
    print("Successfully authenticated.")

    response = response.json()
    print(json.dumps(response, indent=2))
    access_token = response["access_token"]


else:
    print(
        f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")


##### **Prerequisites**
The following prerequisite items set up initial conditions and should be run before the draft use cases below.
- Create a Term Type
- Create a Term
- Create a Parent Term

##### Create a Term Type
A term type is required for all drafts and terms.

In [ ]:
url = sasserver + "/glossary/termTypes"

payload = json.dumps({
  "name": prefix + " Term Type",
  "description": "Example Term Type with all attribute values populated.",
  "label": "Example Term Type",
  "attributes": [
    {
      "name": "singleLineTextAttribute",
      "label": "A single-line text attribute",
      "description": "This is a description of the attribute. Single-line text attributes can be up to 4000 characters long and cannot contain newline characters.",
      "required": True,
      "type": "single-line",
      "defaultValue": "This is an example of a single-line text attribute."
    },
    {
      "name": "multiLineTextAttribute",
      "label": "A multi-line text attribute",
      "description": "Multi-line text attributes can be up to 4000 characters.  Newline characters are allowed.",
      "type": "multi-line",
      "defaultValue": "This is an example of a multi-line text attribute\\nwith\\nnewline\\ncharacters."
    },
    {
      "name": "booleanAttribute",
      "label": "A boolean attribute",
      "description": "Boolean attributes can be either \"true\" or \"false\".",
      "type": "boolean",
      "defaultValue": "true"
    },
    {
      "name": "dateAttribute",
      "label": "A date attribute",
      "description": "Date attributes should be specified with the format \"YYYY-MM-DD\".",
      "type": "date"
    },
    {
      "name": "timeAttribute",
      "label": "A time attribute",
      "description": "Time attributes should be specified with the format \"hh:mm:ssZ\".",
      "type": "time",
      "defaultValue": "10:23:45Z"
    },
    {
      "name": "dateTimeAttribute",
      "label": "A date-time attribute",
      "description": "Date-time attributes should be specified with the format \"YYYY-MM-DDThh:mm:ssZ\".",
      "type": "date-time",
      "defaultValue": "2024-02-12T10:23:45.000Z"
    },
    {
      "name": "singleSelectAttribute",
      "label": "A single-select attribute",
      "description": "Single-select attributes must specify at least one item in the \"items\" field.",
      "type": "single-select",
      "items": [
        "Item 1",
        "Item 2",
        "Item 3"
      ],
      "defaultValue": "Item 1"
    }
  ]
})

headers = {
  'Content-Type': 'application/vnd.sas.glossary.term.type+json',
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("POST", url, headers=headers, data=payload, verify=False)

if response.status_code == 201:
    print("Successfully created term type.")
    response = response.json()
    print(json.dumps(response, indent=2))
    term_type_id = response["id"]
else:
    print(
        f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")

##### Create a Term
This term is used in a later step.

In [ ]:
url = sasserver + "/glossary/terms?publish=true"

payload = json.dumps({
  "name": prefix + " Term",
  "label": "Created Term",
  "description": "Term will be used in a later step",
  "termTypeId": term_type_id,
  "attributes": {
    "singleLineTextAttribute": "NC",
    "multiLineTextAttribute": "NC",
    "singleSelectAttribute": "Item 1",
    "booleanAttribute": True,
    "dateAttribute": "2024-05-24",
    "timeAttribute": "10:00:00Z",
    "dateTimeAttribute": "2024-05-23T12:00:00Z"
  }
})
headers = {
  'Accept': 'application/vnd.sas.glossary.term+json',
  'Content-Type': 'application/vnd.sas.glossary.term+json',
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("POST", url, headers=headers, data=payload, verify=False)

if response.status_code == 201:
    print("Successfully created term.")
    response = response.json()
    print(json.dumps(response, indent=2))
    termId = response["id"]
else:
    print(f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")

##### Create a Parent Term
This term is used as the parent to two of the drafts that are created in the following steps.

In [ ]:
url = sasserver + "/glossary/terms?publish=true"

payload = json.dumps({
  "name": prefix + " Parent Term",
  "label": "Created Parent Term",
  "description": "This term will be used as the parent of the two drafts created in the following steps",
  "termTypeId": term_type_id,
  "attributes": {
    "singleLineTextAttribute": "NC",
    "multiLineTextAttribute": "NC",
    "singleSelectAttribute": "Item 1",
    "booleanAttribute": True,
    "dateAttribute": "2024-05-24",
    "timeAttribute": "10:00:00Z",
    "dateTimeAttribute": "2024-05-23T12:00:00Z"
  }
})
headers = {
  'Accept': 'application/vnd.sas.glossary.term+json',
  'Content-Type': 'application/vnd.sas.glossary.term+json',
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("POST", url, headers=headers, data=payload, verify=False)

if response.status_code == 201:
    print("Successfully created term.")
    response = response.json()
    print(json.dumps(response, indent=2))
    parentTermId = response["id"]
else:
    print(f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")

##### Create a Draft
This use case creates a new draft.

In [ ]:
url = sasserver + "/glossary/terms?publish=false"

payload = json.dumps({
  "name": prefix + " Draft",
  "label": "Created Draft",
  "description": "This is a created draft",
  "termTypeId": term_type_id,
  "attributes": {
    "singleLineTextAttribute": "NC",
    "multiLineTextAttribute": "NC",
    "singleSelectAttribute": "Item 1",
    "booleanAttribute": True,
    "dateAttribute": "2024-05-24",
    "timeAttribute": "10:00:00Z",
    "dateTimeAttribute": "2024-05-23T12:00:00Z"
  }
})
headers = {
  'Accept': 'application/vnd.sas.glossary.term+json',
  'Content-Type': 'application/vnd.sas.glossary.term+json',
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("POST", url, headers=headers, data=payload, verify=False)

if response.status_code == 201:
    print("Successfully created draft.")
    draftEtag = response.headers["ETag"]
    response = response.json()
    print(json.dumps(response, indent=2))
    draftId = response["id"]
else:
    print(f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")

##### Create a Draft with a Parent Term
This use case creates a draft with a parent term.

In [ ]:
url = sasserver + "/glossary/terms?publish=false"

payload = json.dumps({
  "name": prefix + " Child Draft",
  "label": "Created Draft",
  "description": "This is a child draft of a term",
  "termTypeId": term_type_id,
  "parentId": parentTermId,
  "attributes": {
    "singleLineTextAttribute": "NC",
    "multiLineTextAttribute": "NC",
    "singleSelectAttribute": "Item 1",
    "booleanAttribute": True,
    "dateAttribute": "2024-05-24",
    "timeAttribute": "10:00:00Z",
    "dateTimeAttribute": "2024-05-23T12:00:00Z"
  }
})
headers = {
  'Accept': 'application/vnd.sas.glossary.term+json',
  'Content-Type': 'application/vnd.sas.glossary.term+json',
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("POST", url, headers=headers, data=payload, verify=False)

if response.status_code == 201:
    print("Successfully created draft.")
    response = response.json()
    print(json.dumps(response, indent=2))
    childDraftId = response["id"]
else:
    print(f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")

##### Create a Child Draft of a Draft
This use case creates a draft as the child of the draft previously made.

In [ ]:
url = sasserver + "/glossary/terms?publish=false"

payload = json.dumps({
  "name": prefix + " Grandchild Draft",
  "label": "Created Draft",
  "description": "This is a child draft of a draft",
  "termTypeId": term_type_id,
  "parentId": childDraftId,
  "attributes": {
    "singleLineTextAttribute": "NC",
    "multiLineTextAttribute": "NC",
    "singleSelectAttribute": "Item 1",
    "booleanAttribute": True,
    "dateAttribute": "2024-05-24",
    "timeAttribute": "10:00:00Z",
    "dateTimeAttribute": "2024-05-23T12:00:00Z"
  }
})
headers = {
  'Accept': 'application/vnd.sas.glossary.term+json',
  'Content-Type': 'application/vnd.sas.glossary.term+json',
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("POST", url, headers=headers, data=payload, verify=False)

if response.status_code == 201:
    print("Successfully created draft.")
    grandchildDraftEtag = response.headers["ETag"]
    response = response.json()
    print(json.dumps(response, indent=2))
    grandchildDraftId = response["id"]
else:
    print(f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")

##### Create a Draft From an Existing Term
This use casae creates a draft that is associated with the specified term.

In [ ]:
url = sasserver + "/glossary/terms/" + termId + "/draft"

payload = json.dumps({
  "id": termId,
  "name": prefix + " TermDraft",
  "label": "Created Draft",
  "description": "This is a draft created from a term",
  "termTypeId": term_type_id,
  "attributes": {
    "singleLineTextAttribute": "NC",
    "multiLineTextAttribute": "NC",
    "singleSelectAttribute": "Item 1",
    "booleanAttribute": True,
    "dateAttribute": "2024-05-24",
    "timeAttribute": "10:00:00Z",
    "dateTimeAttribute": "2024-05-23T12:00:00Z"
  }
})
headers = {
  'Accept': 'application/vnd.sas.glossary.term+json',
  'Content-Type': 'application/vnd.sas.glossary.term+json',
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("POST", url, headers=headers, data=payload, verify=False)

if response.status_code == 201:
    print("Successfully created draft.")
    response = response.json()
    print(json.dumps(response, indent=2))
    termDraftId = response["id"]
else:
    print(f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")

##### Publish a Draft
This use case publishes the draft with the specified ID. This process creates a new term or updates an existing term that is associated with the draft.

In [ ]:
url = sasserver + "/glossary/terms/" + draftId + "/draft/state?action=publish"

payload = ""
headers = {
  'Content-Type': 'application/vnd.sas.glossary.term+json',
  'If-Match': draftEtag ,
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("PUT", url, headers=headers, data=payload, verify=False)

if response.status_code == 200:
    print("Successfully published draft.")
    response = response.json()
    print(json.dumps(response, indent=2))
else:
    print(f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")

##### Get a Draft by ID
This use case demonstrates retrieving a draft by its ID.

In [ ]:
url = sasserver + "/glossary/terms/" + grandchildDraftId + "/draft"

payload = {}
headers = {
  'Accept': 'application/vnd.sas.glossary.term+json',
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("GET", url, headers=headers, data=payload, verify=False)

if response.status_code == 200:
    print("Successfully retrieved draft.")
    response = response.json()
    print(json.dumps(response, indent=2))
else:
    print(f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")


##### Get Drafts and Terms Via Filter
This use case uses a filter to query a subset of terms and drafts.

In [ ]:
url = sasserver + "/glossary/terms?allowDrafts=all&filter=contains(name,'Draft')"

payload = {}
headers = {
  'Accept': 'application/vnd.sas.collection+json',
  'Accept-Item': 'application/vnd.sas.glossary.term+json',
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("GET", url, headers=headers, data=payload, verify=False)

if response.status_code == 200:
    print("Successfully retrieved drafts and terms.")
    response = response.json()
    print(json.dumps(response, indent=2))
else:
    print(f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")


##### Update a Draft
This use case demonstrates updating an existing draft.

In [ ]:
url = sasserver + "/glossary/terms/" + grandchildDraftId + "/draft"

payload = json.dumps({
  "id": grandchildDraftId,
  "name": prefix + " Updated Draft",
  "label": "update Draft",
  "description": "This is a updated draft",
  "termTypeId": term_type_id,
  "parentId": childDraftId,
  "attributes": {
    "singleLineTextAttribute": "NC",
    "multiLineTextAttribute": "NC",
    "singleSelectAttribute": "Item 1",
    "booleanAttribute": True,
    "dateAttribute": "2024-05-24",
    "timeAttribute": "10:00:00Z",
    "dateTimeAttribute": "2024-05-23T12:00:00Z"
  }
})
headers = {
  'Accept': 'application/vnd.sas.glossary.term+json',
  'Content-Type': 'application/vnd.sas.glossary.term+json',
  'If-Match': grandchildDraftEtag,
  'Authorization': 'Bearer ' + access_token

}

response = requests.request("PUT", url, headers=headers, data=payload, verify=False)

if response.status_code == 200:
    print("Successfully updated draft.")
    grandchildDraftEtag = response.headers["ETag"]
    response = response.json()
    print(json.dumps(response, indent=2))
else:
    print(f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")

##### Patch a Draft
This use case changes the Parent ID of a draft using patch op: add method. 
This use case removes the parentId. 

In [ ]:
url = sasserver + "/glossary/terms/" + grandchildDraftId + "/draft"

payload = json.dumps([
  {
    "op": "add",
    "path": "/parentId",
    "value": ""
  }
])
headers = {
  'Accept': 'application/vnd.sas.glossary.term+json',
  'Content-Type': 'application/json-patch+json',
  'If-Match': grandchildDraftEtag,
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("PATCH", url, headers=headers, data=payload)

if response.status_code == 200:
    grandchildDraftEtag = response.headers["ETag"]
    response = response.json()
    print("Successfully patched draft.")

else:
    print(
        f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")

##### Delete a Draft
Use an ID to delete a draft. This request clears all of the content pertaining to the draft.

In [ ]:
url = sasserver + "/glossary/terms/" + termDraftId + "/draft"

payload = {}
headers = {
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("DELETE", url, headers=headers, data=payload, verify=False)

if response.status_code == 204:
    print("Successfully deleted draft.")
else:
    print(f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")

##### Clean Up Drafts, Terms and Term Types
This deletes the drafts, terms, and term types that were created by the use cases.

In [ ]:
url = sasserver + "/glossary/terms?allowDrafts=all&filter=contains(name,'" + prefix + "')&sortBy=creationTimeStamp:descending"

payload = {}
headers = {
  'Accept': 'application/vnd.sas.collection+json',
  'Accept-Item': 'application/vnd.sas.glossary.term+json',
  'Authorization': 'Bearer ' + access_token
}

# Retrieve terms and drafts
response = requests.request("GET", url, headers=headers, data=payload, verify=False)
instances = response.json().get('items', [])

if response.status_code != 200:
    print(f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")
else :
    host = sasserver
    token = access_token
    # Loop and delete all
    for index, a_term in enumerate(instances):
        del_link = next(link['href'] for link in a_term['links'] if link['rel'] == 'delete')

        del_request_url = host + del_link
        del_request_headers = {'Authorization': 'Bearer ' + token}

        response = requests.delete(del_request_url, headers=del_request_headers, verify=False)
        if response.status_code == 204:
            print('Deleted term with id:', a_term['id'])
        else:
            print('Error deleting term:', response.text)

    print('All terms have been processed.')
    # Delete term type
    url = sasserver + f"/glossary/termTypes/{term_type_id}"
    payload = {}
    headers = {
    'Accept': 'application/vnd.sas.glossary.term.type+json',
    'Authorization': 'Bearer ' + access_token
    }

    response = requests.request("DELETE", url, headers=headers, data=payload, verify=False)

    if response.status_code == 204:
        print("Successfully deleted the term type.")
    else:
        print(f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")

